In [3]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [8]:
X = df.iloc[:,:-1].values 
y = df.iloc[:,-1].values

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [25]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout

In [11]:
model = Sequential()

model.add(Dense(32, activation = 'relu', input_dim = 8))
# model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation ='sigmoid'))

model.compile(optimizer ='Adam', loss ='binary_crossentropy', 
              metrics = ['accuracy'])

In [12]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test), verbose = False)

In [13]:
max(history.history['val_accuracy'])

0.7337662577629089

In [14]:
import kerastuner as kt

## Finding best activation function using kerastuner

In [33]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = hp.Choice('optimizer', ['adam', 'sgd','rmsprop', 'adadelta']), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [34]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)

In [35]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

In [36]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [24]:
model = tuner.get_best_models(num_models = 1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [27]:
hisotry = model.fit(X_train, y_train, batch_size = 32, initial_epoch = 6, epochs = 100, validation_data = (X_test, y_test), verbose = False)

In [15]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value = 8,max_value = 128, step = 8)
    print(units)
    model.add(Dense(units = units, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = hp.Choice('optimizer', ['adam', 'sgd','rmsprop', 'adadelta']), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [16]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)

8


In [17]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6363636255264282

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 00m 08s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'units': 24, 'optimizer': 'adam'}

In [19]:
model = tuner.get_best_models(num_models = 1)[0]

24


In [20]:
hisotry = model.fit(X_train, y_train, batch_size = 32, initial_epoch = 6, epochs = 100, validation_data = (X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 4.6936 - accuracy: 0.6531 - val_loss: 3.5459 - val_accuracy: 0.6818
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 3.0836 - accuracy: 0.6580 - val_loss: 2.6873 - val_accuracy: 0.6883
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 2.5303 - accuracy: 0.6515 - val_loss: 2.4357 - val_accuracy: 0.6883
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 2.2582 - accuracy: 0.6515 - val_loss: 2.1349 - val_accuracy: 0.7013
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 1.9406 - accuracy: 0.6596 - val_loss: 1.8713 - val_accuracy: 0.7013
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 1.7046 - accuracy: 0.6629 - val_loss: 1.7021 - val_accuracy: 0.6818
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 1.5121 - accuracy: 0.6629 - val_loss: 1.5212 - val_accuracy: 0.6818
Epoch 14

In [22]:
max(history.history['val_accuracy'])

0.7337662577629089

In [28]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72, activation = 'relu', input_dim = 8))
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        model.add(Dense(72, activation = 'relu'))
        
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [29]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'my_dir', project_name= 'Tuner')

In [30]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6558441519737244
Total elapsed time: 00h 00m 13s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [32]:
model = tuner.get_best_models(num_models = 1)[0]

In [33]:
hisotry = model.fit(X_train, y_train, batch_size = 32, initial_epoch = 6, epochs = 100, validation_data = (X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 15ms/step - loss: 0.6268 - accuracy: 0.6808 - val_loss: 0.6412 - val_accuracy: 0.6558
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6249 - accuracy: 0.6466 - val_loss: 0.6721 - val_accuracy: 0.6169
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6007 - accuracy: 0.6840 - val_loss: 0.7888 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6097 - accuracy: 0.6808 - val_loss: 0.6223 - val_accuracy: 0.6623
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5969 - accuracy: 0.6726 - val_loss: 0.6866 - val_accuracy: 0.6429
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6041 - accuracy: 0.6710 - val_loss: 0.6306 - val_accuracy: 0.5974
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5881 - accuracy: 0.6531 - val_loss: 0.6206 - val_accuracy: 0.6688
Epoch 14

In [32]:
def build_model(hp):
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        
        if counter == 0:
            model.add(Dense(units = hp.Int('units'+str(i),min_value = 8, max_value = 128, step = 8), 
                            activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'softmax']), input_dim = 8))
            model.add(Dropout(hp.Choice('dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(units = hp.Int('units'+str(i),min_value = 8, max_value = 128, step = 8), 
                            activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid','softmax']))) 
            model.add(Dropout(hp.Choice('dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
        
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer', values = ['adam', 'rmsprop', 'sgd','nadam']), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [33]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'my_dir', project_name= 'final')

In [34]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 07s


In [36]:
tuner.get_best_hyperparameters(num_trials = 1)[0].values

{'num_layers': 6,
 'units0': 72,
 'activation': 'softmax',
 'dropout0': 0.5,
 'optimizer': 'nadam',
 'units1': 16,
 'dropout1': 0.1,
 'units2': 8,
 'dropout2': 0.1,
 'units3': 8,
 'dropout3': 0.1,
 'units4': 8,
 'dropout4': 0.1,
 'units5': 8,
 'dropout5': 0.1}

In [38]:
model = tuner.get_best_models(num_models = 1)[0]

In [39]:
hisotry = model.fit(X_train, y_train, batch_size = 32, initial_epoch = 5, epochs = 100, validation_data = (X_test, y_test))

Epoch 6/100
20/20 [==============================] - 3s 14ms/step - loss: 0.6540 - accuracy: 0.6531 - val_loss: 0.6569 - val_accuracy: 0.6429
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6536 - accuracy: 0.6531 - val_loss: 0.6557 - val_accuracy: 0.6429
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6520 - accuracy: 0.6531 - val_loss: 0.6547 - val_accuracy: 0.6429
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6498 - accuracy: 0.6531 - val_loss: 0.6539 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6518 - accuracy: 0.6531 - val_loss: 0.6532 - val_accuracy: 0.6429
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6506 - accuracy: 0.6531 - val_loss: 0.6530 - val_accuracy: 0.6429
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6473 - accuracy: 0.6531 - val_loss: 0.6526 - val_accuracy: 0.6429
Epoch 13/